In [ ]:
# ===============================
# 0️⃣ Importações e Preparação
# ===============================
import os
import random
import shutil
from google.colab import files
from IPython.display import Image, display
import glob
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Bibliotecas carregadas!")

# Pasta base
base_path = '/content/FarmTech_Fase6'
folders = [
    'images/train','images/val','images/test',
    'labels/train','labels/val','labels/test'
]
for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)
print("Estrutura de pastas criada!")

# ===============================
# 1️⃣ Upload Imagens e Labels
# ===============================
def upload_files(msg):
    print(msg)
    uploaded = files.upload()
    return list(uploaded.keys())

# Upload das imagens
copos_imgs = upload_files("Suba suas 40 imagens de COPOS")
garrafas_imgs = upload_files("Suba suas 40 imagens de GARRAFAS")

# Upload dos labels .txt
copos_lbls = upload_files("Suba os arquivos .TXT de COPOS")
garrafas_lbls = upload_files("Suba os arquivos .TXT de GARRAFAS")

# ===============================
# 2️⃣ Organizar dataset (train/val/test)
# ===============================
def split_and_move(files_list, labels_list, class_name):
    combined = list(zip(files_list, labels_list))
    random.shuffle(combined)
    files_list, labels_list = zip(*combined)

    train_files = files_list[:32]
    val_files = files_list[32:36]
    test_files = files_list[36:40]

    train_labels = labels_list[:32]
    val_labels = labels_list[32:36]
    test_labels = labels_list[36:40]

    def move(files, labels, img_folder, lbl_folder):
        for f,l in zip(files,labels):
            shutil.move(f, os.path.join(img_folder, f))
            shutil.move(l, os.path.join(lbl_folder, l))

    move(train_files, train_labels, os.path.join(base_path,'images/train'), os.path.join(base_path,'labels/train'))
    move(val_files, val_labels, os.path.join(base_path,'images/val'), os.path.join(base_path,'labels/val'))
    move(test_files, test_labels, os.path.join(base_path,'images/test'), os.path.join(base_path,'labels/test'))

# Organizar copos e garrafas
split_and_move(copos_imgs, copos_lbls, 'copo')
split_and_move(garrafas_imgs, garrafas_lbls, 'garrafa')
print("Dataset dividido em treino, val e teste!")

# ===============================
# 3️⃣ Reorganizar para CNN
# ===============================
base_img = '/content/FarmTech_Fase6/images'
classes = ['copo','garrafa']

def reorganize_for_cnn(split):
    img_path = os.path.join(base_img, split)
    for cls in classes:
        cls_folder = os.path.join(img_path, cls)
        os.makedirs(cls_folder, exist_ok=True)
    # mover imagens
    for f in os.listdir(img_path):
        if f.lower().endswith('.jpg'):
            moved = False
            for cls in classes:
                if cls in f.lower():
                    shutil.move(os.path.join(img_path,f), os.path.join(img_path,cls,f))
                    moved = True
                    break
            if not moved:
                print(f"Atenção: não foi possível identificar classe da imagem {f}")

for split in ['train','val','test']:
    reorganize_for_cnn(split)
print("Pastas reorganizadas para CNN!")

# ===============================
# 4️⃣ Instalar YOLOv5
# ===============================
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt
%cd ..

# ===============================
# 5️⃣ Criar data.yaml para YOLOv5
# ===============================
data_yaml = f"""
train: /content/FarmTech_Fase6/images/train
val: /content/FarmTech_Fase6/images/val
test: /content/FarmTech_Fase6/images/test

nc: 2
names: ['copo','garrafa']
"""

with open('data.yaml','w') as f:
    f.write(data_yaml)
print("Arquivo data.yaml criado!")

# ===============================
# 6️⃣ Treinar YOLOv5
# ===============================
%cd yolov5
!python train.py --img 640 --batch 16 --epochs 30 --data ../data.yaml --weights yolov5s.pt --project runs/train --name exp_30ep
!python train.py --img 640 --batch 16 --epochs 60 --data ../data.yaml --weights yolov5s.pt --project runs/train --name exp_60ep
%cd ..

# ===============================
# 7️⃣ Inferência YOLOv5
# ===============================
!python yolov5/detect.py --weights yolov5/runs/train/exp_30ep/weights/best.pt --source /content/FarmTech_Fase6/images/test --save-txt --project yolov5/runs/detect --name test_exp_30
!python yolov5/detect.py --weights yolov5/runs/train/exp_60ep/weights/best.pt --source /content/FarmTech_Fase6/images/test --save-txt --project yolov5/runs/detect --name test_exp_60

# Mostrar imagens detectadas
print("Imagens detectadas (exp_30):")
for img_path in glob.glob('yolov5/runs/detect/test_exp_30/images/*.jpg'):
    display(Image(filename=img_path))
print("Imagens detectadas (exp_60):")
for img_path in glob.glob('yolov5/runs/detect/test_exp_60/images/*.jpg'):
    display(Image(filename=img_path))

# ===============================
# 8️⃣ Treinar CNN do zero
# ===============================
img_size = (128,128)
batch_size = 16

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    '/content/FarmTech_Fase6/images/train', target_size=img_size, batch_size=batch_size, class_mode='binary'
)
val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    '/content/FarmTech_Fase6/images/val', target_size=img_size, batch_size=batch_size, class_mode='binary'
)
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    '/content/FarmTech_Fase6/images/test', target_size=img_size, batch_size=1, class_mode='binary'
)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(train_gen, validation_data=val_gen, epochs=30)

# ===============================
# 9️⃣ Avaliação CNN
# ===============================
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

test_steps = len(test_gen)
preds = model.predict(test_gen, steps=test_steps)
y_pred = (preds > 0.5).astype(int).reshape(-1)
y_true = test_gen.classes

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)


Bibliotecas carregadas!
Estrutura de pastas criada!
Suba suas 40 imagens de COPOS


Saving copo (1).jpg to copo (1).jpg
Saving copo (2).jpg to copo (2).jpg
Saving copo (3).jpg to copo (3).jpg
Saving copo (4).jpg to copo (4).jpg
Saving copo (5).jpg to copo (5).jpg
Saving copo (6).jpg to copo (6).jpg
Saving copo (7).jpg to copo (7).jpg
Saving copo (8).jpg to copo (8).jpg
Saving copo (9).jpg to copo (9).jpg
Saving copo (10).jpg to copo (10).jpg
Saving copo (11).jpg to copo (11).jpg
Saving copo (12).jpg to copo (12).jpg
Saving copo (13).jpg to copo (13).jpg
Saving copo (14).jpg to copo (14).jpg
Saving copo (15).jpg to copo (15).jpg
Saving copo (16).jpg to copo (16).jpg
Saving copo (17).jpg to copo (17).jpg
Saving copo (18).jpg to copo (18).jpg
Saving copo (19).jpg to copo (19).jpg
Saving copo (20).jpg to copo (20).jpg
Saving copo (21).jpg to copo (21).jpg
Saving copo (22).jpg to copo (22).jpg
Saving copo (23).jpg to copo (23).jpg
Saving copo (24).jpg to copo (24).jpg
Saving copo (25).jpg to copo (25).jpg
Saving copo (26).jpg to copo (26).jpg
Saving copo (27).jpg to copo (

Saving garrafa (1).jpeg to garrafa (1).jpeg
Saving garrafa (2).jpeg to garrafa (2).jpeg
Saving garrafa (3).jpg to garrafa (3).jpg
Saving garrafa (4).jpg to garrafa (4).jpg
Saving garrafa (5).jpg to garrafa (5).jpg
Saving garrafa (6).jpg to garrafa (6).jpg
Saving garrafa (7).jpg to garrafa (7).jpg
Saving garrafa (8).jpg to garrafa (8).jpg
Saving garrafa (9).jpg to garrafa (9).jpg
Saving garrafa (10).jpg to garrafa (10).jpg
Saving garrafa (11).jpg to garrafa (11).jpg
Saving garrafa (12).jpg to garrafa (12).jpg
Saving garrafa (13).jpg to garrafa (13).jpg
Saving garrafa (14).jpg to garrafa (14).jpg
Saving garrafa (15).jpg to garrafa (15).jpg
Saving garrafa (16).jpg to garrafa (16).jpg
Saving garrafa (17).jpg to garrafa (17).jpg
Saving garrafa (18).jpg to garrafa (18).jpg
Saving garrafa (19).jpg to garrafa (19).jpg
Saving garrafa (20).jpg to garrafa (20).jpg
Saving garrafa (21).jpg to garrafa (21).jpg
Saving garrafa (22).jpg to garrafa (22).jpg
Saving garrafa (23).jpg to garrafa (23).jpg
Sa

Saving copo (1).txt to copo (1).txt
Saving copo (2).txt to copo (2).txt
Saving copo (3).txt to copo (3).txt
Saving copo (4).txt to copo (4).txt
Saving copo (5).txt to copo (5).txt
Saving copo (6).txt to copo (6).txt
Saving copo (7).txt to copo (7).txt
Saving copo (8).txt to copo (8).txt
Saving copo (9).txt to copo (9).txt
Saving copo (10).txt to copo (10).txt
Saving copo (11).txt to copo (11).txt
Saving copo (12).txt to copo (12).txt
Saving copo (13).txt to copo (13).txt
Saving copo (14).txt to copo (14).txt
Saving copo (15).txt to copo (15).txt
Saving copo (16).txt to copo (16).txt
Saving copo (17).txt to copo (17).txt
Saving copo (18).txt to copo (18).txt
Saving copo (19).txt to copo (19).txt
Saving copo (20).txt to copo (20).txt
Saving copo (21).txt to copo (21).txt
Saving copo (22).txt to copo (22).txt
Saving copo (23).txt to copo (23).txt
Saving copo (24).txt to copo (24).txt
Saving copo (25).txt to copo (25).txt
Saving copo (26).txt to copo (26).txt
Saving copo (27).txt to copo (

Saving garrafa (1).txt to garrafa (1).txt
Saving garrafa (2).txt to garrafa (2).txt
Saving garrafa (3).txt to garrafa (3).txt
Saving garrafa (6).txt to garrafa (6).txt
Saving garrafa (7).txt to garrafa (7).txt
Saving garrafa (8).txt to garrafa (8).txt
Saving garrafa (9).txt to garrafa (9).txt
Saving garrafa (10).txt to garrafa (10).txt
Saving garrafa (11).txt to garrafa (11).txt
Saving garrafa (12).txt to garrafa (12).txt
Saving garrafa (13).txt to garrafa (13).txt
Saving garrafa (14).txt to garrafa (14).txt
Saving garrafa (15).txt to garrafa (15).txt
Saving garrafa (16).txt to garrafa (16).txt
Saving garrafa (17).txt to garrafa (17).txt
Saving garrafa (18).txt to garrafa (18).txt
Saving garrafa (19).txt to garrafa (19).txt
Saving garrafa (20).txt to garrafa (20).txt
Saving garrafa (21).txt to garrafa (21).txt
Saving garrafa (22).txt to garrafa (22).txt
Saving garrafa (23).txt to garrafa (23).txt
Saving garrafa (24).txt to garrafa (24).txt
Saving garrafa (25).txt to garrafa (25).txt
Sa

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Caminho da pasta onde o YOLO salva os resultados (ajuste se necessário)
result_folder = 'yolov5/runs/train/exp'  # exp, exp2, etc.

# Lê os logs do treinamento
results_path = os.path.join(result_folder, 'results.csv')
results = pd.read_csv(results_path)

# Exibe as primeiras linhas para ver o conteúdo
print("📄 Dados disponíveis no log:")
print(results.head())

# Cria os gráficos
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(results['epoch'], results['train/box_loss'], label='Box Loss')
plt.plot(results['epoch'], results['train/obj_loss'], label='Obj Loss')
plt.plot(results['epoch'], results['train/cls_loss'], label='Class Loss')
plt.title('Treinamento - Perdas')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(results['epoch'], results['metrics/precision(B)'], label='Precisão (P)')
plt.plot(results['epoch'], results['metrics/recall(B)'], label='Recall (R)')
plt.title('Precisão e Recall por Época')
plt.xlabel('Época')
plt.ylabel('Valor')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(results['epoch'], results['metrics/mAP50(B)'], label='mAP@50')
plt.plot(results['epoch'], results['metrics/mAP50-95(B)'], label='mAP@50-95')
plt.title('Desempenho do Modelo (mAP)')
plt.xlabel('Época')
plt.ylabel('Valor')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(results['epoch'], results['val/box_loss'], label='Val Box Loss')
plt.plot(results['epoch'], results['val/obj_loss'], label='Val Obj Loss')
plt.plot(results['epoch'], results['val/cls_loss'], label='Val Cls Loss')
plt.title('Validação - Perdas')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
